In [24]:
import pandas as pd
import numpy as np
import re
import scipy
import fancyimpute as fi

In [25]:
most_matches = pd.read_csv('most_seasons_unnormalized.csv')
recent_matches = pd.read_csv('recent_seasons_unnormalized.csv')

most_matches = most_matches.fillna(np.nan)
most_matches = most_matches.replace(-1, np.nan)

recent_matches = recent_matches.fillna(np.nan)
recent_matches = recent_matches.replace(-1, np.nan)

most_matches.drop(list(most_matches.filter(regex='Unnamed: ')), axis=1, inplace=True)
recent_matches.drop(list(recent_matches.filter(regex='Unnamed: ')), axis=1, inplace=True)

all_matches = most_matches.append(recent_matches, ignore_index=True)

In [26]:
betting_odds_home = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
betting_odds_draw = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
betting_odds_away = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
betting_odds_all = [betting_odds_home, betting_odds_draw, betting_odds_away]
betting_odds_combined = betting_odds_home + betting_odds_draw + betting_odds_away

rows = all_matches[pd.isnull(all_matches[betting_odds_combined]).any(axis=1)]

In [27]:
# for index, row in rows.iterrows():
#     if index % 1000 == 0:
#         print(index)
#     for betting_odds in betting_odds_all:
#         mean = np.mean(row[betting_odds])
#         all_matches.loc[index, row[betting_odds].index[row[betting_odds].isnull().tolist()]] = mean


In [28]:
home = all_matches.filter(regex='__home_')
all_matches[home.columns] = home

In [22]:
home_filled = fi.KNN().fit_transform(home)

Imputing row 1/16566 with 1 missing, elapsed time: 49.186
Imputing row 101/16566 with 1 missing, elapsed time: 49.224
Imputing row 201/16566 with 1 missing, elapsed time: 49.260
Imputing row 301/16566 with 1 missing, elapsed time: 49.279
Imputing row 401/16566 with 1 missing, elapsed time: 49.292
Imputing row 501/16566 with 1 missing, elapsed time: 49.310
Imputing row 601/16566 with 1 missing, elapsed time: 49.328
Imputing row 701/16566 with 0 missing, elapsed time: 49.342
Imputing row 801/16566 with 0 missing, elapsed time: 49.342
Imputing row 901/16566 with 0 missing, elapsed time: 49.342
Imputing row 1001/16566 with 0 missing, elapsed time: 49.342
Imputing row 1101/16566 with 1 missing, elapsed time: 49.352
Imputing row 1201/16566 with 1 missing, elapsed time: 49.369
Imputing row 1301/16566 with 1 missing, elapsed time: 49.387
Imputing row 1401/16566 with 1 missing, elapsed time: 49.412
Imputing row 1501/16566 with 1 missing, elapsed time: 49.430
Imputing row 1601/16566 with 1 missi

Imputing row 14101/16566 with 1 missing, elapsed time: 51.317
Imputing row 14201/16566 with 1 missing, elapsed time: 51.348
Imputing row 14301/16566 with 1 missing, elapsed time: 51.427
Imputing row 14401/16566 with 1 missing, elapsed time: 51.458
Imputing row 14501/16566 with 1 missing, elapsed time: 51.525
Imputing row 14601/16566 with 1 missing, elapsed time: 51.593
Imputing row 14701/16566 with 1 missing, elapsed time: 51.629
Imputing row 14801/16566 with 1 missing, elapsed time: 51.663
Imputing row 14901/16566 with 1 missing, elapsed time: 51.711
Imputing row 15001/16566 with 1 missing, elapsed time: 51.769
Imputing row 15101/16566 with 1 missing, elapsed time: 51.823
Imputing row 15201/16566 with 1 missing, elapsed time: 51.888
Imputing row 15301/16566 with 0 missing, elapsed time: 51.950
Imputing row 15401/16566 with 0 missing, elapsed time: 51.950
Imputing row 15501/16566 with 0 missing, elapsed time: 51.951
Imputing row 15601/16566 with 0 missing, elapsed time: 51.951
Imputing

In [23]:
home_filled = pd.DataFrame(data=home_filled, columns=all_matches.columns, index=all_matches.index)
home_filled[-10:]

ValueError: Shape of passed values is (21, 16566), indices imply (72, 16566)

In [ ]:
cols_to_normalize = ['__home_buildUpPlaySpeed', '__home_buildUpPlayDribbling',
                     '__home_buildUpPlayPassing',
                     '__home_chanceCreationPassing', '__home_chanceCreationCrossing',
                     '__home_chanceCreationShooting', '__home_defencePressure',
                     '__home_defenceAggression', '__home_defenceTeamWidth',
                     '__away_buildUpPlaySpeed', 
                     '__away_buildUpPlayDribbling', '__away_buildUpPlayPassing',
                     '__away_chanceCreationPassing', '__away_chanceCreationCrossing',
                     '__away_chanceCreationShooting', '__away_defencePressure',
                     '__away_defenceAggression', '__away_defenceTeamWidth']

all_matches_filled_normed = all_matches_filled.copy()

# Normalize columns
for column in cols_to_normalize:
    col = np.array(all_matches_filled[column].values.astype(np.float))
    all_matches_filled_normed[column] = (col - col.min()) / (col.max() - col.min())

In [ ]:
most_matches_filled = all_matches_filled[:len(most_matches)]
most_matches_filled_normed = all_matches_filled_normed[:len(most_matches)]

recent_matches_filled = all_matches_filled[-len(recent_matches):]
recent_matches_filled_normed = all_matches_filled_normed[-len(recent_matches):]

In [ ]:
most_matches_filled.to_csv('most_seasons_filled_unnormalized.csv', index=False)
recent_matches_filled.to_csv('recent_seasons_filled_unnormalized.csv', index=False)

most_matches_filled_normed.to_csv('most_seasons_filled_normalized.csv', index=False)
recent_matches_filled_normed.to_csv('recent_seasons_filled_normalized.csv', index=False)